# **LTI&trade; Advantage** bootcamp notebook

## Introduction

The notebook shows how to interact with the LTI Advantage ecosystem from a tool implementer viewpoint. It interacts with an actual test server which has been built as a platform simulator to support this notebook.


## Setup

Here we just import the libraries that will be needed in this notebook, define some utility functions and constants.

In [56]:
import requests
import json
import jwt
import base64
from cryptography.hazmat.primitives.asymmetric.rsa import RSAPublicNumbers
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.serialization import Encoding, PublicFormat
from IPython.display import display, Markdown

def decode_int(b64value):
    return int.from_bytes(base64.urlsafe_b64decode(b64value), byteorder='big')

# This notebook queries an actual test server. It needs its location:
platform_url='http://localhost:5000'

First we need to get a new tool deployment from the server for this notebook instance to use.
Each tool must have a client_id and a private key that will be used to interact with the platform services and send messages back to the platform.
It also needs the keyset URL that exposes the platform public keys needed to validate the incoming messages.

While this information is required for each tool, how it is obtained by the tool is NOT currently part of the LTI specifications.

In [68]:
r = requests.get(platform_url + '/newtool')
tool_info = json.loads(r.text)

display(Markdown('### Tool information'))
display(Markdown('Here is the tool information generated for this notebook. It is stored in ```tool_info``` variable.'))
display(Markdown('```json\n'+ json.dumps(tool_info, indent=4)+'```'))

### Tool information

Here is the tool information generated for this notebook. It is stored in ```tool_info``` variable.

```json
{
    "client_id": "8",
    "deployment_id": "deployment_8",
    "key": {
        "e": "AQAB",
        "kty": "RSA",
        "use": "sig",
        "alg": "RS256",
        "d": "FSrvlWGsm-c2Xx3MtH6vgSqu2yYAglCTYu_1z5TUA6mT8e-tqACKa31Bu4dga6OdR3458E7huJU_tElmYU2njQ_oyBctFh1HZa5I_oCOvM87GrHaWpyW5RKGidCqky6ZcJKlihzUNlWQhUT9wfeCZ11wcNABzROxCXXv2KDMqljZ0m_KgsMZcEXW3xc4RVmzvE_6LhwbZDzgaL6cBnf1VwQ8U5ESu1yxZyENHYO-8gg_fy1qttkyqfEIoZDmWtq1Vzzmyj3UDj7YzYOPjO404Qzt5awpZKX3pxDs_dUTGb26K7O5m-BUGI_gdYjjeCmqkv7rnE47y1cTjiGLqDnqOQ=="
    }
}```

## Deep Linking - Creating a Link
This section will use the deep linking specification to create a Resource Link to the platform. That resource link will be gradable and used in the following sections of that notebook.

[Deep Linking](https://www.imsglobal.org/specs/lticiv1p0) is a **UI flow**, it is an important piece that is sometimes missed on 1st reading. The user is redirected from the platform to the tool to pick or create one or multiple piece of content (often, LTI links), and the the tool redirects the UI back to the tool with the actual selection (or an empty payload if nothing was picked or created).

So there are 2 messages:

1. `ContentItemRequestSelection` from the platform to the tool to start the picking/create session. This is a typical platform launch that contains the context and the user information, and what kind of content items may be created in this flow (for example, this flow might indicate it only wants LTI links).
1. `ContentItemSelection` from the tool back to the plaform using the `content_item_return_url` provided in the request.

Once a tool is added to a course, usually the first launch from the platform will be a Deep Linking request.

![deep linking](assets/lti_advantage_deeplinking.png "LTI Deep Link")

### Setup: Getting a Deep Linking Request

This notebook is not a tool actually launched, so the test platform as way to give us the token that it
would include in an actual request, so we can build a mock POST parameter `post_data`

In [74]:
r = requests.get("{}/tool/{}/cisr".format(platform_url, tool_info['client_id']))

post_data = {
    'id_token':r.text
}

### Task 1: Verify the JWT is properly signed

The first thing before to display to the user the picker/authoring interface to create the link is to validate this request is properly signed. This is done by decoding the JWT using public key from the platform.
The test platform exposes its keys in a keyset format at a well-know location (.well-known/jwks.json)

In [75]:
# We need the key set now
keyset = json.loads(requests.get(platform_url + '/.well-known/jwks.json').text)

display(Markdown('#### Platform keyset'))
display(Markdown('```json\n'+ json.dumps(keyset, indent=4)+'```'))

platform_keys = {}

# let's transform as a map for ease of use, and just the PEM because this is what is used by JWT lib
for key in keyset['keys']:
    public_key = RSAPublicNumbers(decode_int(key['e']), decode_int(key['n'])).public_key(default_backend())
    pem = public_key.public_bytes(Encoding.PEM, PublicFormat.SubjectPublicKeyInfo)
    platform_keys[key['kid']] = pem
    


#### Platform keyset

```json
{
    "keys": [
        {
            "e": "AQAB",
            "use": "sig",
            "alg": "HS256",
            "kty": "rsa",
            "n": "wb68Yov1n2pe3wrlBsgUGfQJFOe_7oaya-Y45q_yRjMx2YhO5ZER8mhM81OjRnXMdq7tUJxy-PyHmVJFgTGwZAyokNvDmkPA_1ifVv_c0LnJW9C9BTl2ehljUciv1abwOITKEPcQoQj3ISmGCKMEB-eiQMs5mruunb1AalLnJu_TAfl-BnEhBRznd9thyQcoPUrDD2nfsnmVCoMQJaQzl6EBqCI1d-hK4abNHog_KTMpoe3LrSnUiIjtPWkvWNFv0j1h07HlqhOcWQ6fGHxwLEb1phXhb2xlQ79sIvrJp58cERyMrp6VDb8ozVCwBcgfBE8Eqww8gBCyrMgZtYeflQ==",
            "kid": "1517151491_0"
        },
        {
            "e": "AQAB",
            "use": "sig",
            "alg": "HS256",
            "kty": "rsa",
            "n": "0dqZ5VuLqhjOuW7_YFTUn_dogYbwZhJJdafHyvCdwIdGaMrn6E9dkgx_4jSTonCDhn2LZOWVKn-6Es8pW8gI2S3vjIqxe3CryP-fkCR-Y5ZRG9hUjktIE_suoURTk2VbAmYAyXpV-iECn8Rr0KkFfo6Te4H_VU21VPjXBEVANIxTaXtze3dI-6hhtjYnB2KPcgj5PvDAbQh6IytHcVWHxjACYbVXyb0uQvpRwsE5PsDa2P9ZxxFmr6npyuwM9RblyIF7CDSU6ZrNu-ZPBucdkTy78DCedUPmo6dqZa5t1GeNawZJtLIlj_y696C_sARuFsNGIzBvfleWIsoFMYO10w==",
            "kid": "1517151491_1"
        },
        {
            "e": "AQAB",
            "use": "sig",
            "alg": "HS256",
            "kty": "rsa",
            "n": "q0lkmWCrMLrB7icb1xvYf1vzKZxmn9Ez5imo2pXPRry7KJdbq3EVrgJ1QClV1mo41Lh6alKf4RK8WitI61IgOI1zDLeNtrj0ef-idXiKS5d3M4VVWuQ80w1fXHeB65ebXpEYH9AZlNDjo9jHahfA_8blYCviK8_v68CTpHWnGSNhGHEUBpy__c_YttsiadVjg7Qf7h9ddmZMXifWPgLtrCWOXbuqHOJ-OKvPEpEvM7lBOaSqqSUxuhQTnAPrXsxJCABPNqpoUBVkLNdfxPnTHDWF-X_kf_gsVRPHuOhUgL-nI3lyoA5l8V4--b-NoWWgI_UzCtGaElI2k6MNEZSdqw==",
            "kid": "1517151491_2"
        },
        {
            "e": "AQAB",
            "use": "sig",
            "alg": "HS256",
            "kty": "rsa",
            "n": "29AVqbzp1wXw3s4TFod3cPUo7DNWcWh0NnnSC_Z8ZoZSbz3-9iTP1w_hgwqknALCbtRe0Xz4j9htjEjyA8XdzlPq-B2Ik3mGINT6Ya5AyZaLO7dQ4fEBYUP6v3TuTXhW9z-RYxDoRj3L7mvGvJpU0uqCPJhiZ-pat7truZ82LqAuDkjnJLorQxYnJtlrflBkUKSMixW9-OfByPMMJNnZ0gRHOwMne0m6pyrG_5LNw4EFVwS9X5EQ7tedTJ7ndFVN7GuN5Us7uUC-zWZheoOwXDPqVANksmPGy0U5wV9huS51_qUdzOdGnGrnWgtWSgeS2MtisrFoNCi2ObbNfuCYPw==",
            "kid": "1517151491_3"
        }
    ]
}```

In [76]:
# Let's get the kid so we can get the proper public key
encoded_jwt = post_data['id_token']
jwt_headers = jwt.get_unverified_header(encoded_jwt)

content_item_message = jwt.decode(encoded_jwt, 
                                  platform_keys[jwt_headers['kid']], 
                                  jwt_headers['alg'],
                                  audience = tool_info['client_id'])

display(Markdown('#### Decoded ContentItemSelectionRequest message'))
display(Markdown('```json\n'+ json.dumps(content_item_message, indent=4)+'```'))


#### Decoded ContentItemSelectionRequest message

```json
{
    "iss": "http://localhost:5000",
    "http://imsglobal.org/lti/version": "1.3.0",
    "sub": "LTIBCU_11",
    "iat": 1517185658,
    "email": "Amos.Thad@example.com",
    "family_name": "Thad",
    "given_name": "Amos",
    "http://imsglobal.org/lti/message_type": "ContentItenSelectionRequest",
    "nonce": "3599c204-048b-11e8-a22c-c48e8ffb7857",
    "http://imsglobal.org/lti/deployment_id": "deployment_8",
    "aud": "8",
    "exp": 1517185718,
    "http://imsglobal.org/lti/tool_platform": {
        "guid": "ltibc_at_1517151492",
        "name": "LTI Bootcamp Platform"
    },
    "name": "Amos Thad"
}```